# Analysis of Volatility Instruments

In [18]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime as dt
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import yfinance as yf

import hvplot
import hvplot.pandas
import matplotlib.pyplot as plt


# Imports from internal packages
import sys

module_path1 = str(Path("../vixcoin_functions/"))
if module_path1 not in sys.path:
    sys.path.append(module_path1)
from feature_functions import (retrieve_yahoo_close, 
                               retrieve_yahoo_volume,
                               retrieve_close_multiple_tickers
)

In [19]:
# Tickers of the VIX, and volatility securities to analyze
ticker_list= ["^VIX", "VIXM", "VXX","UVXY", "spy"]

print(f"Current number of tickers: {len(ticker_list)}")


Current number of tickers: 5


In [24]:
# Retrive data of close prices for ETNs
close_prices_df = retrieve_close_multiple_tickers(ticker_list)

print("Completed retrieve of close prices")

Processing Close ^VIX
Processing Close VIXM
Processing Close VXX
Processing Close UVXY
Processing Close spy
Completed retrieve of close prices


In [22]:
close_prices_df

,^VIX,VIXM,VXX,UVXY,spy
Date,,,,,
2011-02-01,17.629999,274.839996,NaN,NaN,105.733864
2011-02-02,17.299999,274.239990,NaN,NaN,105.531685
2011-02-03,16.690001,272.600006,NaN,NaN,105.766228
2011-02-04,15.930000,269.239990,NaN,NaN,106.065475
2011-02-07,16.280001,265.839996,NaN,NaN,106.728630
...,...,...,...,...,...
2021-11-10,18.730000,30.549999,21.600000,16.620001,461.996887
2021-11-11,17.660000,30.150000,21.139999,16.120001,462.146362
2021-11-12,16.290001,29.940001,20.360001,15.240000,465.634094


Obs: We see that the VIXM is the one with the longest history. That is a characteristics that we like, because the longer the history, the longer the training.

In [35]:
# Plot returns of VIX Index, VIX Futurtes ETNs, and S&P500 ETF (SPY)
returns=close_prices_df.pct_change()

returns.hvplot(
    title="Return Time Series of the VIX Index and several ETNs that Trade vIX Futures"

)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [36]:
# Plot returns of VIX Index and SPY

returns.hvplot(y=["^VIX", "spy"],
    title="Return Time Series of the VIX Index and the SPY ETF"

)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [30]:
# Summary Statistics of VIX Index, ETFs, and SPY ETF.
returns.describe()

,^VIX,VIXM,VXX,UVXY,spy
count,2717.000000,2717.000000,960.000000,2547.000000,2717.000000
mean,0.003250,-0.000597,-0.000594,-0.004441,0.000604
std,0.084733,0.020836,0.049939,0.077811,0.010640
min,-0.269583,-0.140413,-0.155574,-0.334499,-0.109424
25%,-0.043936,-0.011963,-0.028254,-0.045922,-0.003379
50%,-0.006309,-0.001975,-0.008006,-0.011468,0.000728
75%,0.036630,0.008439,0.016054,0.025275,0.005453
max,1.155979,0.182147,0.370602,0.662064,0.090603


Obs: We can see that 50% of the times the return of VIXM is negative, which is risky. Hopefully a ML model can help us in skewing the trade days to more positive returns.


For seasonality Analysis, please refer the notebook *vixm_seasonality_prophet.ipynb* in this same folder

In [83]:
# Summary Statistics by day of the week
dayofweek=["Mon","Tue","Wed","Thu","Fri"]
returns_vixm_dayofweek=pd.DataFrame(columns=dayofweek)
print(returns_vixm_dayofweek)

for day in range(0,5):
    #print (f"{dayofweek[day]}")
    #print(returns_vixm_df[returns_vixm_df.index.dayofweek==day].describe())
    #print(returns_vixm_df[returns_vixm_df.index.dayofweek==day].reset_index()["VIXM"])
    returns_vixm_dayofweek[dayofweek[day]]=returns_vixm_df[returns_vixm_df.index.dayofweek==day].reset_index()["VIXM"] 
    #print(returns_vixm_dayofweek[dayofweek[day]])

returns_vixm_dayofweek.describe()



Empty DataFrame
Columns: [Mon, Tue, Wed, Thu, Fri]
Index: []


,Mon,Tue,Wed,Thu,Fri
count,510.000000,510.000000,510.000000,510.000000,510.000000
mean,-0.001279,-0.000520,0.000276,-0.000160,-0.000976
std,0.023867,0.019611,0.021628,0.020390,0.019604
min,-0.106364,-0.140413,-0.062656,-0.089400,-0.059891
25%,-0.015158,-0.011791,-0.010623,-0.010332,-0.012106
50%,-0.003089,-0.000549,-0.001686,-0.001926,-0.002302
75%,0.009617,0.009217,0.007267,0.008430,0.007845
max,0.182147,0.097572,0.175966,0.138069,0.098617


We can see that Wed is the only day when VIXM has a positive average return. These are the days of the rolling of contracts.
Also, on Tuesdays and Fridays the maximum returns are significantly smaller than on Monday and Wednesday.

In [84]:
returns_vixm_dayofweek.hvplot(
    kind='box',
    width=1000,
    height=800,
    title= "Box Plot for Return of VIXM ETN for Different Days of the Week "
)

:BoxWhisker   [Variable]   (value)

The Box plot shows that Friday is the day with smallder negative returns.
In conclusion, based on this plots, there is no evidence of an strong seasonality based on day of the week. However a deeper analysis will be done using Facebook Prophet.

In [94]:
# Correlations
returns.corr()
correlations = returns.corr()
correlations.hvplot.heatmap(
        width = 1300,
        cmap ='RdYlBu', 
        title = "Correlation Matrix of thr VIX Index, Several ETNs and the S&P500 Index"
)


:HeatMap   [columns,index]   (value)

# Observations on correlations
The higher correlation with VIX is UVXY and VXX, which has short history. However, VXX has a high correlation with VIXM, which has a longer history. 
Then a good start is to invest in VIXM, and in the future use a similar model to invest in VXX and UVXY.  